In [1]:
import numpy as np 
import pandas as pd 
import plotly.express as px
from matplotlib import pyplot as plt


import seaborn as sns
import numpy.matlib
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from scipy import stats
from scipy.stats import norm
from joblib import Parallel, delayed
import shutil
import glob
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
from sklearn.metrics import mean_squared_error
np.random.seed(0)
import os
import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from scipy.stats import norm, skew
# train

all_data = pd.read_csv('../data/feature_data_530.csv')
# glove
glove_tags = pd.read_csv('../data/alltags_feature.csv')
glove_title = pd.read_csv('../data/title_feature.csv')
# glove_title = pd.read_csv('../data/title_feature.csv')
all_data = pd.concat([all_data, glove_tags, glove_title], axis=1)
columns = ['Title_len', 'Title_number', 'Alltags_len', 'Alltags_number', 'photo_count', 'totalTags', 'totalGeotagged', 'totalFaves',
          'totalInGroup','photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']
skew_features = all_data[columns].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[abs(skew_features) > 0.75]
skew_index = high_skew.index
for i in skew_index:
    all_data[i] = np.log1p(all_data[i])
    
useless_columns = ['Pid','mean_label'] 
useless_columns += ['user_fe_{}'.format(i) for i in range(399)]
useless_columns += ['loc_fe_{}'.format(i) for i in range(400)]
all_data = all_data.drop(useless_columns, axis=1)


train_all_data = all_data[all_data['train_type'] != -1]
submit_all_data = all_data[all_data['train_type'] == -1]
feature_columns = ['train_type','label']
all_data = all_data.drop(feature_columns, axis=1)

train_all_data = train_all_data.reset_index(drop=True)
submit_all_data = submit_all_data.reset_index(drop=True)


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## extracr BLIP  scores of the text_image pairs


In [2]:
train_label_df = train_all_data[['label']]
train_feature_df = train_all_data.drop(feature_columns, axis=1)

submit_label_df = submit_all_data[['label']]
submit_feature_df = submit_all_data.drop(feature_columns, axis=1)

print(len(train_feature_df), len(submit_feature_df), len(train_feature_df.columns))
print(len(train_label_df), len(submit_label_df), len(train_feature_df.columns))
cate_cols = ['Uid', 'Category', 'Subcategory', 'Concept', 'Mediatype', 'hour', 'day', 'weekday', 'week_hour', 'year_weekday','Geoaccuracy', 'ispro' , 'Ispublic']

305613 180581 682
305613 180581 682


In [3]:
# !pip install optuna

In [4]:
# import numpy as np 
# import pandas as pd

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")
import gc

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA
# from umap import UMAP
from sklearn.cluster import KMeans
from haversine import haversine

import optuna
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

bold = ['\033[1m', '\033[0m']

In [16]:
# pip install lightgbm --install-option=--gpu --install-option="--boost-root=C:/local/boost_1_69_0" --install-option="--boost-librarydir=C:/local/boost_1_69_0/lib64-msvc-14.1"

/usr/lib/python3/dist-packages/pip/_internal/commands/install.py:255: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Note: you may need to restart the kernel to use updated packages.


In [21]:
# !pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"

/usr/lib/python3/dist-packages/pip/_internal/commands/install.py:255: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)


In [27]:
# !apt install cmake

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cmake-data libarchive13 libcurl4 libjsoncpp1 librhash0 libuv1
Suggested packages:
  cmake-doc ninja-build lrzip
The following NEW packages will be installed:
  cmake cmake-data libarchive13 libcurl4 libjsoncpp1 librhash0 libuv1
0 upgraded, 7 newly installed, 0 to remove and 58 not upgraded.
Need to get 6113 kB of archives.
After this operation, 30.2 MB of additional disk space will be used.
Do you want to continue? [Y/n] ^C


In [26]:
!cmake --DUSE_CUDA=1

/bin/bash: cmake: command not found


In [5]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def f_importance_plot(f_imp):
    fig = plt.figure(figsize = (15, 0.35*len(f_imp)))
    plt.title('Feature importances', size=25, y=1.05, 
              fontname='Calibri', fontweight='bold', color='#444444')
    a = sns.barplot(data=f_imp, x='avg_imp', y='feature', 
                    palette='Blues_d', linestyle="-", 
                    linewidth=1, edgecolor="black")
    plt.xlabel('')
    plt.xticks([])
    plt.ylabel('')
    plt.yticks(size=11, color='#444444')
    
    for j in ['right', 'top', 'bottom']:
        a.spines[j].set_visible(False)
    for j in ['left']:
        a.spines[j].set_linewidth(0.5)
    plt.show()

In [11]:
!cmake -DUSE_GPU=1

CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.


CMake Error at CMakeLists.txt:101 (include):
  include could not find requested file:

    CompileFlags.cmake


CMake Error: File /MMChallenge/SMP/HyFea-main/Utilities/std/cmSTL.hxx.in does not exist.
CMake Error at CMakeLists.txt:112 (configure_file):
  configure_file Problem configuring file


CMake Error: File /MMChallenge/SMP/HyFea-main/.clang-tidy does not exist.
CMake Error at CMakeLists.txt:307 (configure_file):
  configure_file Problem configuring file


CMake Error at CMakeLists.txt:739 (include):
  include could not find requested file:

    /MMChallenge/SMP/HyFea-main/Tests/CMakeInstall.cmake


CMake Error at CMakeLists.txt:346 (include):
  include could not find requested file:

    Utilities/cmThirdPartyChecks.cmake
Call Stack (most recent call first):
  CMakeList

In [7]:



import xgboost as xgb
import lightgbm as lgb
from scipy import stats
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
lgb_params = {
    'max_depth': 9,
    'learning_rate': 0.01,
    # 'min_data_in_leaf': 36, 
    # 'num_leaves': 100, 
    # 'feature_fraction': 0.8, 
    # 'bagging_fraction': 0.89, 
    # 'bagging_freq': 5, 
    # 'lambda_l2': 28,
    # 'seed': seed,
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'device': 'gpu', 
    # 'gpu_platform_id': 0,
    'gpu_device_id': 0,
    'n_jobs': -1,
    'metric': 'rmse',
    'verbose': -1
}

f_imp = pd.DataFrame({'feature': train_feature_df.columns})
predictions, scores = np.zeros(len(submit_feature_df)), []

k = KFold(n_splits=5, random_state=2020, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(k.split(train_feature_df, train_label_df)):
    print(f'\n--- FOLD {fold+1} ---')
        
    lgb_train = lgb.Dataset(data=train_feature_df.iloc[train_idx], 
                            label=train_label_df.iloc[train_idx],
                            categorical_feature=cate_cols)
    lgb_valid = lgb.Dataset(data=train_feature_df.iloc[val_idx], 
                            label=train_label_df.iloc[val_idx],
                            categorical_feature=cate_cols,
                            reference=lgb_train)

    model = lgb.train(params=lgb_params, 
                      train_set=lgb_train, 
                      num_boost_round=50000,
                      valid_sets=[lgb_train, lgb_valid], 
                      valid_names=['train', 'val'],
                      verbose_eval=False,
                      callbacks=[lgb.log_evaluation(1000),
                                 lgb.early_stopping(1000, verbose=False)])
    
    f_imp['fold_'+str(fold+1)] = model.feature_importance()
    b_itr = model.best_iteration
    
    val_preds = model.predict(train_feature_df.iloc[val_idx], num_iteration=b_itr)
    val_score = rmse(train_label_df['label'].iloc[val_idx], val_preds)
    val_mae = mean_squared_error(train_label_df['label'].iloc[val_idx], val_preds)
    val_src = stats.spearmanr(train_label_df['label'].iloc[val_idx], val_preds)[0]
    scores.append(val_score)
    print("valid: MSE: %.4f, MAE: %.4f, SRC: %.4f"%(val_score, val_mae,val_src))
    predictions += model.predict(submit_feature_df, num_iteration=b_itr) / 5
    print(f'--- RMSE: {bold[0]}{round(val_score, 6)}{bold[1]} | best iteration: {bold[0]}{b_itr}{bold[1]} ---')
    
    del lgb_train, lgb_valid, val_preds, val_score, model
    gc.collect()

print('*'*45)
print(f'Mean RMSE: {bold[0]}{round(np.mean(scores), 6)}{bold[1]}')

f_imp['avg_imp'] = f_imp[f_imp.columns[1:]].mean(axis=1)
f_imp.sort_values('avg_imp', ascending=False, inplace=True)
f_importance_plot(f_imp)


--- FOLD 1 ---


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1


LightGBMError: GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1

In [ ]:
lgbm_preds = predictions.copy()
ss['LGBM_pred'] = predictions
ss.to_csv('lgbm_pred.csv', index=False)

In [ ]:
for i in cate_cols:
    train_feature_df[i] = train_feature_df[i].astype("category")
    submit_feature_df[i] = submit_feature_df[i].astype("category")
    

In [ ]:
## use xgboost for regression

xgb_params = {
    'max_depth': 9,
    # 'max_cat_to_onehot':6,
    'eta': 0.01,
    # 'colsample_bytree': 0.66,
    # 'subsample': 0.76,
    # 'min_child_weight': 22,
    'lambda': 16, 
    'gamma': 1,
    'tree_method': 'gpu_hist',
    'booster': 'gbtree',
    'predictor':'gpu_predictor',
    'seed': seed,
    'gpu_id':0,
    'enable_categorical':True,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

f_imp = pd.DataFrame({'feature': train_feature_df.columns})
predictions, scores = np.zeros(len(submit_feature_df)), []

k = KFold(n_splits=5, random_state=seed, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(k.split(train_feature_df, train_label_df)):
    print(f'\n--- FOLD {fold+1} ---')
    
    dtrain = xgb.DMatrix(train_feature_df.iloc[train_idx], label=train_label_df.iloc[train_idx])
    dvalid = xgb.DMatrix(train_feature_df.iloc[val_idx], label=train_label_df.iloc[val_idx])
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

    model = xgb.train(params=xgb_params, 
                      dtrain=dtrain, 
                      num_boost_round=50000,
                      evals=watchlist, 
                      verbose_eval=5000,
                      callbacks=[xgb.callback.EarlyStopping(rounds=1000,
                                                            data_name='eval',
                                                            maximize=False,
                                                            save_best=True)])
    
    fi = model.get_score(importance_type='weight')
    fi = pd.DataFrame({'feature':fi.keys(),f'importance_{fold}':fi.values()})
    f_imp = f_imp.merge(fi, on='feature', how='left').fillna(0)
    b_itr = model.best_ntree_limit
    
    val_preds = model.predict(dvalid)
    val_score = rmse(train_label_df['label'].iloc[val_idx], val_preds)
    val_mae = mean_squared_error(train_label_df['label'].iloc[val_idx], val_preds)
    val_src = stats.spearmanr(train_label_df['label'].iloc[val_idx], val_preds)[0]
    scores.append(val_score)
    print("valid: MSE: %.4f, MAE: %.4f, SRC: %.4f"%(val_score, val_mae,val_src))
    
    predictions += model.predict(xgb.DMatrix(submit_feature_df)) / 5
    print(f'--- RMSE: {bold[0]}{round(val_score, 6)}{bold[1]} | best iteration: {bold[0]}{b_itr}{bold[1]} ---')
    
    del dtrain, dvalid, watchlist, val_preds, val_score, model
    gc.collect()

print('*'*45)
print(f'Mean RMSE: {bold[0]}{round(np.mean(scores), 6)}{bold[1]}')

f_imp['avg_imp'] = f_imp[f_imp.columns[1:]].mean(axis=1)
f_imp.sort_values('avg_imp', ascending=False, inplace=True)
f_importance_plot(f_imp)

In [ ]:
xgb_preds = predictions.copy()
ss['xgb_pre'] = predictions
ss.to_csv('xgb_pred.csv', index=False)